# Library

In [148]:
import pandas as pd
import numpy as np

from sklearn.linear_model import Lasso, Ridge, ElasticNet, LogisticRegression
from sklearn.svm import SVC, SVR
from sklearn.ensemble import (
    RandomForestRegressor,
    RandomForestClassifier,
    VotingRegressor,
    VotingClassifier,
    StackingRegressor,
    StackingClassifier,
    GradientBoostingRegressor,
    GradientBoostingClassifier,
    BaggingRegressor,
)
from sklearn.tree import DecisionTreeRegressor, DecisionTreeClassifier, plot_tree, export_graphviz

from sklearn.preprocessing import LabelEncoder, StandardScaler, MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.model_selection import RandomizedSearchCV
import graphviz

import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px

# Ensemble

<img src="https://cdn.prod.website-files.com/5d7b77b063a9066d83e1209c/61f7bbd4e90cce440b88ea32_ensemble-learning.png" width="600" height="300"/>

<br>

두 개 이상의 모델을 결합하여 task를 수행하는 기법 <br> 
오차의 분산을 줄이기에 개별 모델보다 강건함(robust) <br>
    -> 한 모델에서 포착하지 못했던 정보를 다른 모델에서 포착할 수 있음 <br>
따라서 모델 간 다양성이 클수록 ensemble 모델의 효과가 커짐 <br>

<br>

<font style="font-size:20px"> 아래와 같은 상황에서 사용 </font> <p>
1. 모델 선택의 어려움: 데이터셋 내의 다양한 부분에서 서로 다른 모델들이 강점을 보이는 경우, 여러 모델을 앙상블하여 각 클래스에 대한 강점을 결합함으로써 전체적인 성능 향상
2. 문제의 복잡성: 단일 분류기로 처리하기 어려운 복잡한 결정 경계를 다룰 때, 여러 분류기의 앙상블은 보다 복잡한 결정 경계 근사
3. 데이터 양: 충분한 데이터가 있는 경우, 단일 분류기보다 여러 분류기를 사용하여 작업을 분할하고 예측 시에 앙상블하는 것이 더 효과적일 수 있음. 반대로, 데이터가 제한적인 경우, 부트스트래핑(데이터의 여러 부분집합에서 다양한 분류기를 훈련) 전략 사용

<br>

<font style="font-size:20px"> 앙상블을 통한 이점 </font> <p>
1. 예측 신뢰도: 앙상블은 각 개별 모델의 신뢰도에 의존하기에, 높은 신뢰도를 가진 모델의 예측을 결합함으로써 단순한 과반수 투표보다 전체적인 예측 신뢰성을 향상
2. 정보 통합: 앙상블은 동일한 클래스에 관련된 다양한 데이터셋이나 조건에서 훈련된 모델들의 예측을 통합하여 분류의 견고성을 향상시키고 편향을 줄임

## Voting

<img src="https://www.researchgate.net/publication/362311832/figure/fig3/AS:11431281078760466@1660240577836/The-figure-shows-with-an-example-how-both-hard-and-soft-voting-work-The-ensemble.ppm" width="600" height="300"/>

모델의 결과를 취합하여 성능 향상을 취하는 방법 <br>
hard voting과 soft voting 두 방법 존재 <br>

<br>

<font style="font-size:20px"> Hard Voting </font> <p>
각 모델에서의 결과를 다수결로 결정하는 방법

<br>

<font style="font-size:20px"> Soft Voting </font> <p>
각 모델에서 결과를 확률로 받은 후 확률의 평균 등을 이용하여 최종 결과 결정

<br>

<font style="font-size:20px"> 사용 방법 </font> <p>

> ```python
> from sklearn.ensemble import VotingRegressor
> 
> voting_regressor = VotingRegressor([
>   ('model1', model1),
>   ('model2', model2),
>   ('model3', model3),
> ])
> 
> voting_regressor.fit(X, y)    # train
> voting_regressor.predict(X)   # predict
> ```

<br>

주요 파라미터
- estimators: base learners
- weights: base learner 결과에 계산할 가중치

In [86]:
def convert_category_into_integer(df: pd.DataFrame, columns: list):
    label_encoders = {}
    for column in columns:
        label_encoder = LabelEncoder()
        df.loc[:, column] = label_encoder.fit_transform(df[column])

        label_encoders.update({column: label_encoder})
    
    return df, label_encoders

def scaler(dfs: list, scaler, non_targer_column: list):
    target_columns = dfs[0].columns.difference(non_targer_column)
    scaler.fit(dfs[0].loc[:, target_columns])
    for data in dfs:
        data.loc[:, target_columns] = scaler.transform(data.loc[:, target_columns])

    return dfs, scaler

In [17]:
diamonds = sns.load_dataset('diamonds')
diamonds, _ = convert_category_into_integer(diamonds, ('cut', 'color', 'clarity'))
diamonds = diamonds.drop_duplicates().reset_index(drop=True)
train, test = train_test_split(diamonds, test_size=0.2, random_state=0)

C:\Users\USER\AppData\Local\Temp\ipykernel_20444\1150649826.py:5: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[2 3 1 ... 4 3 2]' has dtype incompatible with category, please explicitly cast to a compatible dtype first.
  df.loc[:, column] = label_encoder.fit_transform(df[column])
C:\Users\USER\AppData\Local\Temp\ipykernel_20444\1150649826.py:5: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[1 1 1 ... 0 4 0]' has dtype incompatible with category, please explicitly cast to a compatible dtype first.
  df.loc[:, column] = label_encoder.fit_transform(df[column])
C:\Users\USER\AppData\Local\Temp\ipykernel_20444\1150649826.py:5: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[3 2 4 ... 2 3 3]' has dtype incompatible with category, please explicitly cast to a compatible dtype firs

In [21]:
decision_tree = DecisionTreeClassifier(random_state=0)
decision_tree.fit(train.drop(columns=['cut']), train['cut'])

DecisionTreeClassifier(random_state=0)

In [24]:
(decision_tree.predict(test.drop(columns=['cut'])) == test['cut']).mean()

0.708523096942095

In [27]:
# hard voting
voting_classifier = VotingClassifier(
    [(f'model{i+1}', DecisionTreeClassifier(random_state=i)) for i in range(10)]
)
voting_classifier.fit(train.drop(columns=['cut']), train['cut'])
(voting_classifier.predict(test.drop(columns=['cut'])) == test['cut']).mean()

0.713263314434427

In [28]:
# soft voting
voting_classifier = VotingClassifier(
    [(f'model{i+1}', DecisionTreeClassifier(random_state=i)) for i in range(10)],
    voting='soft',
)
voting_classifier.fit(train.drop(columns=['cut']), train['cut'])
(voting_classifier.predict(test.drop(columns=['cut'])) == test['cut']).mean()

0.7131703689934009

In [35]:
# diamonds dataset
# 1. 결측치, 중복값 확인 및 처리
diamonds = sns.load_dataset('diamonds')

# 2. label encoder를 활용하여 범주형 데이터 정수화
diamonds, _ = convert_category_into_integer(diamonds, ('cut', 'color', 'clarity'))

# 3. train, test split (random_state=0)
train, test = train_test_split(diamonds, random_state=0)

# 4. train 데이터로 scaler fit 후 train과 test scaling
# (정수화된 범주형 데이터는 scaling하지 않음)
target_columns = train.columns.difference(['cut', 'color', 'clarity'])
standard_scaler = StandardScaler()
standard_scaler.fit(train.loc[:, target_columns])
train.loc[:, target_columns] = standard_scaler.transform(train.loc[:, target_columns])
test.loc[:, target_columns] = standard_scaler.transform(test.loc[:, target_columns])

# 5. SVC, Logistic Regression 객체 정의
support_vector_classifier = SVC(kernel='linear')
logistic_regression = LogisticRegression(random_state=0, max_iter=1000)

# 6. SVC, logistic regresson 개별적으로 학습 후 각 모델의 성능 확인
support_vector_classifier.fit(train.drop(columns=['cut']), train['cut'])
logistic_regression.fit(train.drop(columns=['cut']), train['cut'])

print('svc acc: ', (support_vector_classifier.predict(
    test.drop(columns=['cut'])) == test['cut']).mean()
)
print('logistic acc: ', (logistic_regression.predict(
    test.drop(columns=['cut'])) == test['cut']).mean()
)

# 7. voting classifier에 사용할 분류기 정의
voting_classifier = VotingClassifier([
    ('svm', SVC(kernel='linear')),
    ('logistic_regression', LogisticRegression(random_state=0, max_iter=1000)),
])

# 8. voting classifier 학습 후 성능 확인
voting_classifier.fit(train.drop(columns=['cut']), train['cut'])

# 9. 개별 모델의 성능과 voting 모델의 성능 비교
print('voting acc: ', (voting_classifier.predict(
    test.drop(columns=['cut'])) == test['cut']).mean()
)

C:\Users\USER\AppData\Local\Temp\ipykernel_20444\1150649826.py:5: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[2 3 1 ... 4 3 2]' has dtype incompatible with category, please explicitly cast to a compatible dtype first.
  df.loc[:, column] = label_encoder.fit_transform(df[column])
C:\Users\USER\AppData\Local\Temp\ipykernel_20444\1150649826.py:5: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[1 1 1 ... 0 4 0]' has dtype incompatible with category, please explicitly cast to a compatible dtype first.
  df.loc[:, column] = label_encoder.fit_transform(df[column])
C:\Users\USER\AppData\Local\Temp\ipykernel_20444\1150649826.py:5: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[3 2 4 ... 2 3 3]' has dtype incompatible with category, please explicitly cast to a compatible dtype firs

svc acc:  0.6760103819058213
logistic acc:  0.6560622914349277
voting acc:  0.6698553948832036


## Stacking

<img src="https://cdn.prod.website-files.com/5d7b77b063a9066d83e1209c/61a4414dba9e9f94d7b31368_RhQ6ctlYepNo3J-yChyk_jLM_siHT9eGIJTpcI0NEPhADEcGic31JW4TWwLLzWv0LvqyDjFx9yQ8m16kKENTtPZeW-fY-9z6k7m-rsmPseGIeHhB-IiI0V5t4hImEPZRnEWPChAo.png" width="600" height="300"/>

<br>
<br>

<img src="https://miro.medium.com/v2/resize:fit:720/format:webp/1*T-JHq4AK3dyRNi7gpn9-Xw.png" width="600" height="300"/>

<br>

base 모델 학습과 meta model 학습의 두 가지 단계를 통하여 ensemble 모델 구축 <br>
base learner는 동일 데이터에서 여러 모델을 학습하거나 혹은 bagging 방법을 통하여 구축 <br>
이렇게 학습한 모델을의 결과를 meta learner라고 하는 모델의 input으로 사용하여 meta learner 학습 <br>

<br>

<font style="font-size:20px"> 사용 방법 </font> <p>

> ```python
> from sklearn.ensemble import StackingRegressor
> 
> base_learners = [
>   ('model1', model1),
>   ('model2', model2),
>   ('model3', model3),
> ]
> meta_learner = model4
> 
> stacking_regressor = StackingRegressor(
>   estimators=base_learners,
>   final_estimator=meta_learner,
> )
> 
> stacking_regressor.fit(X, y)    # train
> stacking_regressor.predict(X)   # predict
> ```

<br>

주요 파라미터
- estimators: base learners
- final_estimator: meta learner
- cv: num of folds (cross-validation)

In [51]:
# penguin dataset
# 1. 결측치, 중복값 확인 및 처리 (penguin data)
penguins = sns.load_dataset('penguins')
penguins = penguins.dropna()
penguins = penguins.drop_duplicates()

# 2. label encoder를 활용하여 범주형 데이터 정수화
penguins, _ = convert_category_into_integer(penguins, ('species', 'island', 'sex'))

# 3. train, test split (random_state=0)
train, test = train_test_split(penguins, test_size=0.3, random_state=0)

# 4. train 데이터로 scaler fit 후 train과 test scaling
# (정수화된 범주형 데이터는 scaling하지 않음)
def scaler(dfs: list, scaler, non_targer_column: list):
    target_columns = dfs[0].columns.difference(non_targer_column)
    scaler.fit(dfs[0].loc[:, target_columns])
    for data in dfs:
        data.loc[:, target_columns] = scaler.transform(data.loc[:, target_columns])

    return dfs

non_target_columns = ['species', 'island', 'sex']
min_max_scaler = MinMaxScaler()

train, test = scaler([train, test], min_max_scaler, non_target_columns)

# 5. SVC, Logistic Regression, Decision Tree 객체 정의 (random_state=0)
support_vector_classifier = SVC()
logistic_regression = LogisticRegression(random_state=0)
decision_tree = DecisionTreeClassifier(random_state=0)

# 6. SVC, logistic regresson, Decision Tree 개별적으로 학습 후 각 모델의 성능 확인
support_vector_classifier.fit(train.drop(columns=['species']), train.species.astype(int))
logistic_regression.fit(train.drop(columns=['species']), train.species.astype(int))
decision_tree.fit(train.drop(columns=['species']), train.species.astype(int))

print('svc acc: ', (support_vector_classifier.predict(
    test.drop(columns=['species'])) == test['species']).mean()
)
print('logistic_regression acc: ', (logistic_regression.predict(
    test.drop(columns=['species'])) == test['species']).mean()
)
print('decision_tree acc: ', (decision_tree.predict(
    test.drop(columns=['species'])) == test['species']).mean()
)

# 7. stacking classifier에 사용할 분류기 정의 (SVC, logistic regresson, Decision Tree)
#    meta learner 정의 (random forest)
meta_learner = RandomForestClassifier(random_state=0)
stacking_classifier = StackingClassifier([
    ('svc', SVC()),
    ('logistic_regression', LogisticRegression(random_state=0)),
    ('decision_tree', DecisionTreeClassifier(random_state=0)),
], final_estimator=meta_learner)

# 8. stacking classifier 학습 후 성능 확인
stacking_classifier.fit(train.drop(columns=['species']), train.species.astype(int))

# 9. 개별 모델의 성능과 stacking 모델의 성능 비교
print('stacking_classifier acc: ', (stacking_classifier.predict(
    test.drop(columns=['species'])) == test['species']).mean()
)

svc acc:  0.99
logistic_regression acc:  0.97
decision_tree acc:  0.98
stacking_classifier acc:  0.99


## Boosting

<img src="https://cdn.prod.website-files.com/5d7b77b063a9066d83e1209c/61a4414d5e568a661fb7896c_mji7xyiAlyQAdxQde14HY1OVvAVzDyyKhDOo4a4bg53_m2OHUvHhMGexaHuHCfKGRVQQlfFlihuodX7LD5hugPgGw8ZzJV4bHjHc648Zr0LyVr2I0i6ciJvJri_OFCuQpOf81xcn.png" width="600" height="300"/>

<br>

데이터 병렬 처리 대신 순차 처리 발생. <br>
주어진 데이터를 첫 번째 모델로 학습. <br>
첫 번째 모델에 의한 오차를 두 번째 모델에서 학습. <br>
두 번째 모델은 첫 번째 모델이 못 맞춘 부분만을 학습. <br>
위의 작업을 반복하여 개별 모델을 학습 후 ensemble하여 최종 모델 구축. <br>

<br>

<font style="font-size:20px"> 대표 알고리즘 </font> <p>
- Adaptive Boosting (AdaBoost)
- XGBoost
- LightGBM
- CatBoost

In [56]:
tips = sns.load_dataset('tips')
print(tips.isna().sum())
print(tips.duplicated().sum())
tips = tips.drop_duplicates()

total_bill    0
tip           0
sex           0
smoker        0
day           0
time          0
size          0
dtype: int64
1


In [ ]:
tips, _ = convert_category_into_integer(tips, ('sex', 'smoker', 'day', 'time'))
train, test = train_test_split(tips, test_size=0.3, random_state=0)

In [ ]:
min_max_scaler = MinMaxScaler()
train, test = scaler(
    (train, test),
    min_max_scaler,
    ('sex', 'smoker', 'day', 'time'),
)

In [70]:
gradient_boosting = GradientBoostingClassifier(
    n_estimators=100,
    learning_rate=0.5,
    max_depth=4,
    random_state=0
)
gradient_boosting.fit(train.drop(columns=['sex']), train.sex)

GradientBoostingClassifier(learning_rate=0.5, max_depth=4, random_state=0)

In [74]:
(gradient_boosting.predict(test.drop(columns=['sex'])) == test.sex).mean()

0.5205479452054794

In [75]:
gradient_boosting.score(test.drop(columns=['sex']), test.sex)

0.5205479452054794

In [113]:
# 삼성전자 dataset
# 삼성전자의 이전 5일의 주가를 가지고 내일의 주가를 맞추려고 한다.
# 1. 데이터 로드 (일자와 종가만 로드)
samsung = pd.read_csv(
    './data/samsung_2023.csv',
    encoding='cp949',
    usecols=['일자', '종가'])

# 2. 일자를 datetime으로 변환하여 index에 입력
samsung.일자 = pd.to_datetime(samsung.일자)
samsung = samsung.sort_values(by=['일자'])
samsung = samsung.set_index('일자')

# 3. 데이터 전처리 (sliding window를 사용하여 데이터를 window size가 6이 되도록 구성)
window_size = 6
slided_data = np.lib.stride_tricks.sliding_window_view(
    samsung.종가.values,
    window_size,
)

# 4. 3에서의 데이터를 DataFrame으로 구성
#    (column과 index에 적절한 값을 대입)
slided_data = pd.DataFrame(
    slided_data,
    columns=['t-4', 't-3', 't-2', 't-1', 't', 't+1'],
    index=samsung.index[window_size-1:]
)

# 5. train, test split (random_state=0)
train, test = train_test_split(
    slided_data,
    test_size=0.3,
    shuffle=False,
)

# 6. train과 test에 standard scaler 적용
standard_scaler = StandardScaler()
(train, test), standard_scaler = scaler((train, test), standard_scaler, [''])

# 7. gradient boosting 모델 적합 (random_state=0)
gradient_boosting = GradientBoostingRegressor(
    n_estimators=50,
    learning_rate=1,
    random_state=0,
)
gradient_boosting.fit(train.drop(columns=['t+1']), train['t+1'])

# 8. mse 계산 및 예측 결과와 실제 결과를 시각화
predicted_train = gradient_boosting.predict(train.drop(columns=['t+1']))
predicted_train = predicted_train*standard_scaler.scale_[-1] + standard_scaler.mean_[-1]
predicted_test = gradient_boosting.predict(test.drop(columns=['t+1']))
predicted_test = predicted_test*standard_scaler.scale_[-1] + standard_scaler.mean_[-1]

ground_truth_train = train['t+1']*standard_scaler.scale_[-1] + standard_scaler.mean_[-1]
ground_truth_test = test['t+1']*standard_scaler.scale_[-1] + standard_scaler.mean_[-1]

# train.loc[:] = standard_scaler.inverse_transform(train)
# test.loc[:] = standard_scaler.inverse_transform(test)

print(f'train mse: {np.sqrt(((predicted_train - ground_truth_train)**2).sum()) / len(train)}')
print(f'test mse: {np.sqrt(((predicted_test - ground_truth_test)**2).sum()) / len(test)}')

train mse: 1.4540119349091774
test mse: 252.62913149249945


C:\Users\USER\AppData\Local\Temp\ipykernel_20444\2940792817.py:15: FutureWarning:

Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[-1.73606189 -1.327913   -1.39993927 -1.37593051 -1.37593051 -1.30390424
 -1.23187796 -1.25588672 -1.39993927 -1.13584293 -1.06381665 -0.67967652
 -0.55963273 -0.39157142 -0.70368528 -1.25588672 -1.06381665 -0.65566776
 -0.58364148 -1.11183417 -1.03980789 -0.75170279 -0.77571155 -0.82372907
 -0.79972031 -0.72769404 -0.96778162 -0.60765024 -0.87174659 -0.84773783
 -0.99179038 -1.23187796 -1.01579914 -1.18386045 -1.37593051 -1.35192175
 -1.30390424 -1.37593051 -1.13584293 -1.327913   -1.42394803 -1.47196555
 -1.6160181  -1.4959743  -1.73606189 -1.54399182 -1.51998306 -1.18386045
 -1.44795679 -1.42394803 -1.23187796 -0.94377286 -0.77571155 -0.99179038
 -0.79972031 -0.84773783 -0.72769404 -0.53562397 -0.75170279 -0.631659
 -0.55963273 -0.94377286 -0.29553638 -0.12747507 -0.07945756 -0.0554488
 -0.03144004 -

In [118]:
temp = pd.DataFrame({
    'predicted': predicted_train,
    'ground_truth': train['t+1'],
})
px.line(
    temp,
    x=temp.index,
    y=['predicted', 'ground_truth']
)

In [110]:
temp = pd.DataFrame({
    'predicted': predicted_test,
    'ground_truth': test['t+1'],
})
px.line(
    temp,
    x=temp.index,
    y=['predicted', 'ground_truth']
)

## Mixture of Experts (MoE)

<img src="https://cdn.prod.website-files.com/5d7b77b063a9066d83e1209c/61a4414da1649f856e553d74_jiy3kw7cLPoe3-DClXGWwEZo4FNQuf3NzqSpelWXN0di_Ydnyz9QnHqGiFAd87pc-WELMSXGKKdw1wqeA5pjSioytSVNXCmU6wdaV3nFUYZjkKKs_deV_XyUjLOfU8K9o5RWu_n4.png" width="600" height="300"/>

여러 전문화된 하위 모델(전문가)를 조합하여 복잡하고 다양한 패턴을 학습하도록 하는 방법. <br>
기존 단일 모델이 모든 것을 잘 하도록 학습하는 방식에서 모델 별로 잘 하는 분야를 두고 전문화된 모델을 학습. <br>
이러한 모델을 조합하여 최종적인 모델을 구축. <br>
입력 데이터에 따라 각 모델(전문가)의 중요도를 산출하는 gating network를 통해 비중 설정 <br>

<br>

최근 자연어 처리에서 각광받는 GPT도 해당 로직을 탑재한 것으로 추정됨

## Bagging

<img src="https://cdn.prod.website-files.com/5d7b77b063a9066d83e1209c/61a4414d28946a3ac3e69ed9_q-FrlRMLk-5nSxZ_3ONlFpu5hQ61PsuAxkusTD1vEX5NqkdH2Ie0u_75rIySTZKXVI4VBxM-AIw3APQvRboG3kv-3l3cA5c5qyMwwTMe2OLXzoAgA051Dqbx7XVfdJaDyNwrSLUf.png" width="600" height="300"/>

<br>

Boostrap AGGregatING의 약자로 부트스트랩을 통한 방법 <br>
전체 집합에서 일부 데이터를 선택하여 subset을 만든 후(복원추출) 개별 모델을 학습 <br>
-> subset을 여러 개 만들어 동시에 학습할 수 있기에 병렬화 가능
최종 예측은 평균 등의 방법으로 산출 <br>
이 때 사용되는 모델은 **모두 동일한 모델**이어야 함

<br>

<font style="font-size:20px"> 대표 알고리즘 </font> <p>
- Random Forest

<br>

<font style="font-size:20px"> 사용 방법 </font> <p>

> ```python
> from sklearn.ensemble import BaggingRegressor
> 
> bagging_regressor = BaggingRegressor(
>   estimator=model,
> )
> 
> bagging_regressor.fit(X, y)    # train
> bagging_regressor.predict(X)   # predict
> ```

<br>

주요 파라미터
- estimator: base model
- n_estimators: base learner의 수
- random_state: 난수 고정을 위한 seed

In [ ]:
bagging_classifier = BaggingClassifier(
    estimator=DecisionTreeClassifier(random_state=0, max_depth=5, min_samples_split=10),
).fit(train.drop(columns=['cut']), train['cut'])

In [ ]:
print(f'train_acc: {(bagging_classifier.predict(train.drop(columns=['cut'])) == train.cut).mean()}')
print(f'test_acc: {(bagging_classifier.predict(test.drop(columns=['cut'])) == test.cut).mean()}')

In [152]:
# 서울시 대기질자료
fine_dust = pd.read_csv('./data/서울시 대기질 자료 제공_2022.csv', encoding='cp949')

# 1. 중복값 확인 후 제거, 결측: ffill
fine_dust = fine_dust.ffill()
fine_dust = fine_dust.sort_values(by=['일시']).reset_index(drop=True)

# 2. 구분에서 평균은 제거
fine_dust = fine_dust.query('구분 != "평균"')

# 3. 미세먼지와 초미세먼지로 데이터 프레임 분할
ultra_fine_dust = pd.DataFrame(fine_dust.drop(columns=['미세먼지(PM10)']))
find_dust = pd.DataFrame(fine_dust.drop(columns=['초미세먼지(PM2.5)']))

# 4. 칼럼 재정의
#   df1: 강남구-미세먼지(pm10), 강동구-미세먼지(pm10), ...
#   df2: 강남구-초미세먼지(pm2.5), 강동구-초미세먼지(pm2.5), ...
fine_dust = fine_dust.pivot(index='일시', columns='구분', values='미세먼지(PM10)')
ultra_fine_dust = ultra_fine_dust.pivot(index='일시', columns='구분', values='초미세먼지(PM2.5)')

# 5. label 생성
#   - row의 평균이 70보다 크면 1, 아니면 0
fine_dust['fine_dust_label'] = np.where(fine_dust.mean(axis=1) > 70, 1, 0)
ultra_fine_dust['ultra_fine_dust_label'] = np.where(ultra_fine_dust.mean(axis=1) > 70, 1, 0)

# 6. 두 개의 데이터 프레임 결합
total_dust = pd.concat([
    fine_dust,
    ultra_fine_dust,
], axis=1)

# 7. label 재생성 
#   - pm10과 pm2.5가 모두 70보다 크면 2, 둘 중에 하나만 70보다 크면 1, 그렇지 않으면 0
total_dust['label'] = total_dust.filter(regex='label').sum(axis=1)
total_dust = total_dust.drop(columns=['fine_dust_label', 'ultra_fine_dust_label'])

# 8. Stacking을 통해 label을 예측하는 모델 구성
#   base_learners: svc, logistic regression, decision tree
#   meta_learner: random forest
train, test = train_test_split(
    total_dust,
    test_size=0.3,
    random_state=0,
    shuffle=False,
)

meta_learner = RandomForestClassifier(random_state=0)
stacking_regressor = StackingClassifier([
    ('svc', SVC()),
    ('logistic_regression', LogisticRegression(random_state=0)),
    ('decision_tree', DecisionTreeClassifier(random_state=0)),
], final_estimator=meta_learner)
stacking_regressor.fit(train.drop(columns=['label']), train.label)
stacking_regressor.score(train.drop(columns=['label']), train.label)

c:\ProgramData\anaconda3\Lib\site-packages\sklearn\linear_model\_logistic.py:469: ConvergenceWarning:

lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression

c:\ProgramData\anaconda3\Lib\site-packages\sklearn\linear_model\_logistic.py:469: ConvergenceWarning:

lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression

c:\ProgramData\anaconda3\Lib\site-packages

0.9988584474885844